In [1]:
import neural_dimensionality_reduction as ndr
import h5py

In [2]:
fname = '../remy.hdf5'

In [3]:
with h5py.File(fname, 'r') as f:
    f.visit(print)

tetrode_0
tetrode_0/marks
tetrode_0/marks_som_1000_200
tetrode_0/marks_som_500_500
tetrode_0/marks_transformed
tetrode_0/positions
tetrode_1
tetrode_1/marks
tetrode_1/marks_som_1000_200
tetrode_1/marks_som_500_500
tetrode_1/marks_som_500_500_likelihood
tetrode_1/marks_transformed
tetrode_1/positions
tetrode_10
tetrode_10/marks
tetrode_10/marks_som_1000_200
tetrode_10/marks_som_500_500
tetrode_10/marks_transformed
tetrode_10/positions
tetrode_11
tetrode_11/marks
tetrode_11/marks_som_1000_200
tetrode_11/marks_som_500_500
tetrode_11/marks_transformed
tetrode_11/positions
tetrode_12
tetrode_12/marks
tetrode_12/marks_som_1000_200
tetrode_12/marks_som_500_500
tetrode_12/marks_transformed
tetrode_12/positions
tetrode_13
tetrode_13/marks
tetrode_13/marks_som_1000_200
tetrode_13/marks_som_500_500
tetrode_13/marks_transformed
tetrode_13/positions
tetrode_14
tetrode_14/marks
tetrode_14/marks_som_1000_200
tetrode_14/marks_som_500_500
tetrode_14/marks_transformed
tetrode_14/positions
tetrode_15
tet

In [4]:
tetrode_id = 4
scheme_name = 'som_500_500'

In [5]:
def get_position_multiunits(tetrode_id, scheme_name):
    position = ndr.data.retrieve_hdf5_file_positions(fname, tetrode_id)
    multiunits = ndr.data.retrieve_hdf5_file(fname, tetrode_id, scheme_name)

    n_spikes, n_dim = multiunits.shape

    multiunits = multiunits.reshape((n_spikes, n_dim, 1))

    print(position.shape)
    print(multiunits.shape)
    
    return multiunits, position

In [6]:
multiunits, position = get_position_multiunits(tetrode_id, scheme_name)

(90925, 1)
(90925, 3, 1)


In [7]:
from replay_trajectory_classification import ClusterlessDecoder
import numpy as np

In [8]:
model_kwargs = {
    'bandwidth': np.array([1.0, 1.0, 1.0, 6.0]) # 1, 2, 3, 4, position
}
decoder_raw = ClusterlessDecoder(model_kwargs=model_kwargs)  

decoder_raw.fit(position, multiunits)

/home/trevortknguyen/repos/dimensionality/replay_trajectory_classification/state_transition.py:10: RuntimeWarning: invalid value encountered in true_divide
  x /= x.sum(axis=1, keepdims=True)


ClusterlessDecoder(infer_track_interior=True,
                   initial_conditions_type='uniform_on_track',
                   model=<class 'replay_trajectory_classification.misc.NumbaKDE'>,
                   model_kwargs={'bandwidth': array([ 1. ,  1. ,  1. , 12.5])},
                   movement_var=0.05,
                   occupancy_kwargs={'bandwidth': array([ 1. ,  1. ,  1. , 12.5])},
                   occupancy_model=<class 'replay_trajectory_classification.misc.NumbaKDE'>,
                   place_bin_size=2.0, position_range=None, replay_speed=40,
                   transition_type='random_walk')

In [9]:
from replay_trajectory_classification.multiunit_likelihood import estimate_multiunit_likelihood

In [11]:
%%time
likelihood = estimate_multiunit_likelihood(
            multiunits[:3000], decoder_raw.place_bin_centers_,
            decoder_raw.joint_pdf_models_, decoder_raw.ground_process_intensities_,
            decoder_raw.occupancy_, decoder_raw.mean_rates_,
            decoder_raw.is_track_interior_.ravel(order='F'))

CPU times: user 1h 14min 36s, sys: 1.81 s, total: 1h 14min 38s
Wall time: 9min 37s


In [ ]:
%%time
preds = decoder_raw.predict(multiunits)

In [ ]:
preds['likelihood'].values

In [ ]:
likelihoods = preds['likelihood'].values

In [ ]:

with h5py.File(fname, 'r+') as f:                                        
    ds_transformed = f.create_dataset(f'tetrode_{tetrode_id}/marks_{scheme_name}_likelihood', likelihoods.shape, dtype='f')
    ds_transformed[:] = likelihoods

In [ ]:
with h5py.File(fname, 'r') as f:                                        
    ds = f[f'tetrode_{tetrode_id}/marks_{scheme_name}_likelihood']
    print(ds[()])
    print(ds.shape)